<a href="https://colab.research.google.com/github/jeffheaton/present/blob/master/youtube/gan/colab_gan_train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!nvidia-smi

Sat Dec 24 17:43:28 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.82.01    Driver Version: 470.82.01    CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
!pip install torch==1.8.1 torchvision==0.9.1
!pip install ninja

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 804.1/804.1 MB 1.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.4/17.4 MB 50.3 MB/s eta 0:00:00
  Attempting uninstall: torch
    Found existing installation: torch 1.11.0
    Uninstalling torch-1.11.0:
      Successfully uninstalled torch-1.11.0
  Attempting uninstall: torchvision
    Found existing installation: torchvision 0.12.0
    Uninstalling torchvision-0.12.0:
      Successfully uninstalled torchvision-0.12.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pytorch-lightning 1.7.7 requires torch>=1.9.*, but you have torch 1.8.1 which is incompatible.
allennlp 2.10.1 requires torch<1.13.0,>=1.10.0, but you have torch 1.8.1 which is incompatible.


In [3]:
!git clone https://github.com/NVlabs/stylegan2-ada-pytorch.git

Cloning into 'stylegan2-ada-pytorch'...
remote: Enumerating objects: 128, done.
remote: Total 128 (delta 0), reused 0 (delta 0), pack-reused 128
Receiving objects: 100% (128/128), 1.12 MiB | 14.35 MiB/s, done.
Resolving deltas: 100% (57/57), done.


In [4]:
import os
os.makedirs('/kaggle/working/Experiments')
os.makedirs('/kaggle/working/Dataset')
os.makedirs('/kaggle/working/Experiments/prl_dataset-auto1-kimg1000-resumecustom')


In [5]:
!python /kaggle/working/stylegan2-ada-pytorch/dataset_tool.py --source /kaggle/input/prl-dataset-pvt/prl_images --dest /root/Datasets/prl_dataset

/kaggle/working/stylegan2-ada-pytorch/dataset_tool.py:205: DeprecationWarning: BOX is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.BOX instead.
  resample = { 'box': PIL.Image.BOX, 'lanczos': PIL.Image.LANCZOS }[resize_filter]
/kaggle/working/stylegan2-ada-pytorch/dataset_tool.py:205: DeprecationWarning: LANCZOS is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.LANCZOS instead.
  resample = { 'box': PIL.Image.BOX, 'lanczos': PIL.Image.LANCZOS }[resize_filter]
100%|███████████████████████████████████████| 1349/1349 [00:23<00:00, 57.93it/s]


In [6]:
%ls /root/Datasets/prl_dataset

00000/  00001/  dataset.json


In [7]:
a=!ls /root/Datasets/prl_dataset/00001
len(a)

349

In [8]:
cd /kaggle/working/Experiments/prl_dataset-auto1-kimg1000-resumecustom

/kaggle/working/Experiments/prl_dataset-auto1-kimg1000-resumecustom


In [9]:
!wget https://gitlab.cs.fau.de/ek50edub/prl/-/raw/main/network-snapshot-000520.pkl -O network-snapshot-000520.pkl

--2022-12-24 17:45:20--  https://gitlab.cs.fau.de/ek50edub/prl/-/raw/main/network-snapshot-000520.pkl
Resolving gitlab.cs.fau.de (gitlab.cs.fau.de)... 131.188.30.94, 2001:638:a000:4130:131:188:30:94
Connecting to gitlab.cs.fau.de (gitlab.cs.fau.de)|131.188.30.94|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 283158144 (270M) [application/octet-stream]
Saving to: ‘network-snapshot-000520.pkl’

network-snapshot-00 100%[===================>] 270.04M  26.8MB/s    in 11s     

2022-12-24 17:45:31 (25.2 MB/s) - ‘network-snapshot-000520.pkl’ saved [283158144/283158144]



In [10]:
%cd /kaggle/working/

/kaggle/working


In [11]:
!zip -r res1.zip /kaggle/working/Experiments

  adding: kaggle/working/Experiments/ (stored 0%)
  adding: kaggle/working/Experiments/prl_dataset-auto1-kimg1000-resumecustom/ (stored 0%)
  adding: kaggle/working/Experiments/prl_dataset-auto1-kimg1000-resumecustom/network-snapshot-000520.pkl (deflated 7%)


In [12]:
%ls /kaggle/working/Experiments

prl_dataset-auto1-kimg1000-resumecustom/


In [13]:
from IPython.display import FileLink
FileLink(r'sample_data.zip')

/kaggle/working/sample_data.zip

The following command can be used to clear out the newly created dataset.  If something goes wrong and you need to clean up your images and rerun the above command, you should delete your partially created dataset directory.

In [14]:
!pip  install click

In [15]:
# Modify these to suit your needs
EXPERIMENTS = "/kaggle/working/Experiments"
NETWORK = "network-snapshot-000520.pkl"
RESUME = os.path.join(EXPERIMENTS, "prl_dataset-auto1-kimg1000-resumecustom", NETWORK)
DATA = "/root/Datasets/prl_dataset/"
SNAP = 10
kig=1000
gpu=2
# Build the command and run it
cmd = f"python /kaggle/working/stylegan2-ada-pytorch/train.py --gpus {gpu} --kimg {kig} --snap {SNAP} --resume {RESUME} --outdir {EXPERIMENTS} --data {DATA}"
!{cmd}


Training options:
{
  "num_gpus": 2,
  "image_snapshot_ticks": 10,
  "network_snapshot_ticks": 10,
  "metrics": [
    "fid50k_full"
  ],
  "random_seed": 0,
  "training_set_kwargs": {
    "class_name": "training.dataset.ImageFolderDataset",
    "path": "/root/Datasets/prl_dataset/",
    "use_labels": false,
    "max_size": 1349,
    "xflip": false,
    "resolution": 256
  },
  "data_loader_kwargs": {
    "pin_memory": true,
    "num_workers": 3,
    "prefetch_factor": 2
  },
  "G_kwargs": {
    "class_name": "training.networks.Generator",
    "z_dim": 512,
    "w_dim": 512,
    "mapping_kwargs": {
      "num_layers": 2
    },
    "synthesis_kwargs": {
      "channel_base": 16384,
      "channel_max": 512,
      "num_fp16_res": 4,
      "conv_clamp": 256
    }
  },
  "D_kwargs": {
    "class_name": "training.networks.Discriminator",
    "block_kwargs": {},
    "mapping_kwargs": {},
    "epilogue_kwargs": {
      "mbstd_group_size": 4
    },
    "channel_base": 16384,
    "channel_max":

In [16]:
%ls /kaggle/working/Experiments/prl_dataset-auto1-kimg1000-resumecustom/


network-snapshot-000520.pkl
